**steps**
1. identify row with headers
2. map headers to apppriate column number
3. read the line above (a) component till total/sub-total
4. remove all rows with null (expect for yeild)
5. add to full_data


In [87]:
import numpy as np 
import pandas as pd

In [88]:
def read_excel_file(file_path):
    file_ext = file_path.split(".")[-1].lower()
    if file_ext == "xlsb":
        try:
            return pd.read_excel(file_path, sheet_name=None, engine="pyxlsb", dtype=str)
        except Exception as e:
            print(f"❌ Error reading {file_path} (XLSB format): {e}")
            return None
    elif file_ext in ["xls", "xlsx"]:
        try:
            return pd.read_excel(file_path, sheet_name=None, dtype=str)
        except Exception as e:
            print(f"❌ Error reading {file_path} (XLS/XLSX format): {e}")
            return None
    else:
        print(f"⚠️ Unsupported file format: {file_path}")
        return None


In [89]:
path = r"C:\\Users\\vaibh\\Documents\\IPYNB_notebooks\\interview\\data\\data\\HDFC Mutual Fund\\Monthly HDFC Banking & Financial Services Fund - 31 January 2025.xlsx"


In [90]:
all_sheets = []

df_raw = read_excel_file(path)
if df_raw is None:
    pass
for sheet_name, sheet_df in df_raw.items():
    # if sheet_name not in sheets_to_avoid:
    all_sheets.append(sheet_df)

        

In [91]:
df = all_sheets[0]

In [92]:
df_raw.keys()

dict_keys(['HDFCBKGFSF'])

In [93]:
df = all_sheets[0]
df = df.dropna(how='all')
df.reset_index(drop=True , inplace = True)
rows = df.fillna("").agg(" ".join , axis = 1)
df = df.iloc[rows[rows.apply(lambda x : "listing on stock exchange" not in x.lower())].index.to_list()]
df.reset_index(drop=True , inplace = True)

In [94]:
def fetch_header_row(df :pd.DataFrame) -> list[str]: 
    rows = df.astype(str).agg(''.join, axis=1)
    idx = rows[rows.apply(lambda x: "instrument" in x.lower())].index.tolist()[0]
    header_row = df.iloc[idx,:].fillna("NULL")
    header_row = [(header_row.iloc[col]) for col in range(header_row.shape[0])]
    return header_row

In [95]:
df.head(10)

,HDFC Banking & Financial Services Fund (An open ended equity scheme investing in Banking and Financial Services Sector),HDFC Banking & Financial Services Fund (An open ended equity scheme investing in Banking and Financial Services Sector).1,HDFC Banking & Financial Services Fund (An open ended equity scheme investing in Banking and Financial Services Sector).2,HDFC Banking & Financial Services Fund (An open ended equity scheme investing in Banking and Financial Services Sector).3,HDFC Banking & Financial Services Fund (An open ended equity scheme investing in Banking and Financial Services Sector).4,HDFC Banking & Financial Services Fund (An open ended equity scheme investing in Banking and Financial Services Sector).5,HDFC Banking & Financial Services Fund (An open ended equity scheme investing in Banking and Financial Services Sector).6,HDFC Banking & Financial Services Fund (An open ended equity scheme investing in Banking and Financial Services Sector).7,HDFC Banking & Financial Services Fund (An open ended equity scheme investing in Banking and Financial Services Sector).8,HDFC Banking & Financial Services Fund (An open ended equity scheme investing in Banking and Financial Services Sector).9,Income,Hybrid
0,Portfolio as on 31-Jan-2025,Portfolio as on 31-Jan-2025,Portfolio as on 31-Jan-2025,Portfolio as on 31-Jan-2025,Portfolio as on 31-Jan-2025,Portfolio as on 31-Jan-2025,Portfolio as on 31-Jan-2025,Portfolio as on 31-Jan-2025,Portfolio as on 31-Jan-2025,Portfolio as on 31-Jan-2025,NaN,NaN
1,NaN,ISIN,Coupon (%),Name Of the Instrument,Industry+ /Rating,Quantity,Market/ Fair Value (Rs. in Lacs.),% to NAV,Yield,~YTC (AT1/Tier 2 bonds),NaN,NaN
2,NaN,EQUITY & EQUITY RELATED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Equity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,|,INE040A01034,NaN,HDFC Bank Ltd.£,Banks,4465000,75849.19,21.25,NaN,NaN,NaN,NaN
5,|,INE090A01021,NaN,ICICI Bank Ltd.,Banks,5105000,63955.44,17.92,NaN,NaN,NaN,NaN
6,|,INE238A01034,NaN,Axis Bank Ltd.,Banks,2237000,22059.06,6.18,NaN,NaN,NaN,NaN
7,|,INE062A01020,NaN,State Bank of India,Banks,2800000,21641.2,6.06,NaN,NaN,NaN,NaN
8,|,INE128S01021,NaN,Five-Star Business Finance Limited,Finance,1618093,12771.61,3.58,NaN,NaN,NaN,NaN
9,|,INE296A01024,NaN,Bajaj Finance Ltd.,Finance,125000,9856.38,2.76,NaN,NaN,NaN,NaN


In [96]:
header_row = fetch_header_row(df)
header_row

['NULL',
 'ISIN',
 'Coupon (%)',
 'Name Of the Instrument',
 'Industry+ /Rating',
 'Quantity',
 'Market/ Fair Value (Rs. in Lacs.)',
 '% to NAV',
 'Yield',
 '~YTC (AT1/Tier 2 bonds)',
 'NULL',
 'NULL']

In [99]:
n_iter = 0
while "NULL" in header_row and n_iter<4:
    start = None
    for i in range(len(header_row)):
        if start == None and header_row[i] != "NULL":
            start = i
            break

    for i in range(start+1 , len(header_row)):
        if header_row[i] != "NULL":
            end = i
            break
    alter1 = df.iloc[:,start:end].fillna("").agg(" ".join,axis = 1)
    alter2 = df.drop(df.columns[start:end],axis = 1)
    df = pd.concat([alter1 , alter2] , axis = 1)

    header_row = fetch_header_row(df)
    n_iter+=1

            
header_row

['ISIN ',
 'Coupon (%)',
 'Name Of the Instrument',
 'Industry+ /Rating',
 'Quantity',
 'Market/ Fair Value (Rs. in Lacs.)',
 '% to NAV',
 'Yield',
 '~YTC (AT1/Tier 2 bonds)',
 'NULL',
 'NULL']

## **Step 2** 

In [100]:
base_headers = ["Name of Instrument","ISIN" , "Industry" , "Yield" , "Quantity" , "Market Value" , "Net Asset Value (NAV)"]

In [101]:
import requests
def generate_embedding(text:str) -> list[float]:
    url = "https://lamhieu-lightweight-embeddings.hf.space/v1/embeddings"
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }
    data = {
        "model": "snowflake-arctic-embed-l-v2.0",
        "input": text
    }

    response = requests.post(url, headers=headers, json=data)
    if response.ok:
        return response.json()["data"][0]["embedding"]
    else:
        raise Exception("No response")

In [102]:
test_header_row=["Company/Issuer/Instrument Name", "Industry/Rating","Quantity","ISIN","Gibbrish" , "Exposure/Market Value", "yeild to Instrument","% to Nav", "Cupon"]

In [103]:
def header_mapper(header_row , * , base_headers = ["Name of Instrument","ISIN" , "Industry" , "Yield" , "Quantity" , "Market Value" , "Net Asset Value (NAV)"]) -> {str:int}:
    import numpy as np
    from sklearn.metrics.pairwise import cosine_similarity

    header_map = dict()

    base_embeddings = np.array([generate_embedding(value) for value in base_headers])
    header_row_embeddings = np.array([generate_embedding(value) for value in header_row])
    # Compute cosine similarity (shape: 5 x 10)
    similarity_matrix = cosine_similarity(base_embeddings, header_row_embeddings)

    # For each base vector, find the index of the most similar header
    most_similar_indices = np.argmax(similarity_matrix, axis=1)

    # Optionally, get the similarity score too
    most_similar_scores = np.max(similarity_matrix, axis=1)

    # Print results
    for i, (idx, score) in enumerate(zip(most_similar_indices, most_similar_scores)):
        print(f"Base vector {i} ie {base_headers[i]} is most similar to header {idx} ie {header_row[idx]} with score {score:.4f}")
        header_map[base_headers[i]] = int(idx)
        
    return header_map

In [104]:
# header_mapper(test_header_row)

In [105]:
header_map = header_mapper(header_row)
header_map

Base vector 0 ie Name of Instrument is most similar to header 2 ie Name Of the Instrument with score 0.9527
Base vector 1 ie ISIN is most similar to header 0 ie ISIN  with score 1.0000
Base vector 2 ie Industry is most similar to header 3 ie Industry+ /Rating with score 0.6091
Base vector 3 ie Yield is most similar to header 7 ie Yield with score 1.0000
Base vector 4 ie Quantity is most similar to header 4 ie Quantity with score 1.0000
Base vector 5 ie Market Value is most similar to header 5 ie Market/ Fair Value (Rs. in Lacs.) with score 0.5944
Base vector 6 ie Net Asset Value (NAV) is most similar to header 6 ie % to NAV with score 0.5839


{'Name of Instrument': 2,
 'ISIN': 0,
 'Industry': 3,
 'Yield': 7,
 'Quantity': 4,
 'Market Value': 5,
 'Net Asset Value (NAV)': 6}

## **step 3**

## **step 4** read all lines in valid ranges

In [ ]:
import pandas as pd
import re 
def check_isin(val):
    s = str(val).lower().strip()
    s = re.sub("[^a-zA-Z0-9]" , "" , s)
    return s.startswith("in") and s[-1] in "0123456789"

def get_valid_periods(df , header_map):

    mask = df.iloc[:, header_map["ISIN"]].apply(check_isin).values

    # Find continuous True periods
    periods = []
    start = None

    for i, val in enumerate(mask):
        if val:
            if start is None:
                start = i
        else:
            if start is not None:
                periods.append((start, i - 1))
                start = None

    # Edge case: last element was True
    if start is not None:
        periods.append((start, len(mask) - 1))

    print("Passing periods:", periods)
    return periods

periods = get_valid_periods(df , header_map)

Passing periods: [(4, 161), (166, 170), (180, 181)]


In [ ]:
full_data = pd.DataFrame(columns= list(header_map.keys()) + ["type" , "scheme" , "amc name"])

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
for (start_idx , end_idx) in periods:
    scheme_name = re.findall(r'\b[a-zA-Z-\\/\s]+\b', df[start_idx-1:start_idx].fillna("").agg(" ".join , axis = 1).iloc[0])[0]
    print("\n",scheme_name)

    for (index , row) in df.iloc[start_idx:end_idx+1].iterrows():
        values = header_map.copy()
        for (key , idx) in header_map.items():
            values[key] = row.iloc[idx]
        print(f"{index} ",end=" , ") # just to keep track

        full_data = pd.concat([full_data , pd.DataFrame([values])],ignore_index=True)
print("sheet over")



 Equity
4  , 5  , 6  , 7  , 8  , 9  , 10  , 11  , 12  , 13  , 14  , 15  , 16  , 17  , 18  , 19  , 20  , 21  , 22  , 23  , 24  , 25  , 26  , 27  , 28  , 29  , 30  , 31  , 32  , 33  , 34  , 35  , 36  , 37  , 38  , 39  , 40  , 41  , 42  , 43  , 44  , 45  , 46  , 47  , 48  , 49  , 50  , 51  , 52  , 53  , 54  , 55  , 56  , 57  , 58  , 59  , 60  , 61  , 62  , 63  , 64  , 65  , 66  , 67  , 68  , 69  , 70  , 71  , 72  , 73  , 74  , 75  , 76  , 77  , 78  , 79  , 80  , 81  , 82  , 83  , 84  , 85  , 86  , 87  , 88  , 89  , 90  , 91  , 92  , 93  , 94  , 95  , 96  , 97  , 98  , 99  , 100  , 101  , 102  , 103  , 104  , 105  , 106  , 107  , 108  , 109  , 110  , 111  , 112  , 113  , 114  , 115  , 116  , 117  , 118  , 119  , 120  , 121  , 122  , 123  , 124  , 125  , 126  , 127  , 128  , 129  , 130  , 131  , 132  , 133  , 134  , 135  , 136  , 137  , 138  , 139  , 140  , 141  , 142  , 143  , 144  , 145  , 146  , 147  , 148  , 149  , 150  , 151  , 152  , 153  , 154  , 155  , 156  , 157  , 158  , 159  , 1

In [ ]:
full_data = full_data.drop_duplicates()

In [ ]:
full_data

,Name of Instrument,ISIN,Industry,Yield,Quantity,Market Value,Net Asset Value (NAV),type,scheme,amc name
0,Infosys Ltd.,INE009A01021,IT - Software,NaN,707726,13303.83,22.290000000000003,NaN,NaN,NaN
1,BHARTI AIRTEL LTD.,INE397D01024,Telecom - Services,NaN,444377,7226.900000000001,12.11,NaN,NaN,NaN
2,TATA CONSULTANCY SERVICES LTD.,INE467B01029,IT - Software,NaN,125972,5180.47,8.68,NaN,NaN,NaN
3,Tech Mahindra Ltd.,INE669C01036,IT - Software,NaN,285579,4781.88,8.01,NaN,NaN,NaN
4,Zomato Ltd,INE758T01015,Retailing,NaN,1245016,2743.39,4.6,NaN,NaN,NaN
5,HCL Technologies Ltd.,INE860A01027,IT - Software,NaN,141107,2434.73,4.08,NaN,NaN,NaN
6,Wipro Ltd.,INE075A01022,IT - Software,NaN,777882,2426.21,4.07,NaN,NaN,NaN
7,Mphasis Ltd,INE356A01018,IT - Software,NaN,65522,1879.14,3.15,NaN,NaN,NaN
8,Persistent Systems Limited,INE262H01021,IT - Software,NaN,27776,1675.6100000000001,2.81,NaN,NaN,NaN
9,SWIGGY LTD,INE00H001014,Retailing,NaN,376378,1566.1100000000001,2.62,NaN,NaN,NaN


In [ ]:
class A:
    def one(self):
        print("Hello")
        return 1


In [ ]:
aa = A()
def two(a,b):
    print(f"{a+b} two")
    return 2
aa.one = two

In [ ]:
aa.one(1,2)

3 two


2

In [ ]:
header_row

['Name of Instrument',
 'ISIN Code',
 'Industry',
 'Yield',
 'Quantity',
 'Market Value (Rs.in Lacs)',
 '% to Net Assets']

In [ ]:
start = 0
for i in range(len(header_row)):
    if header_row[i] != "NULL":
        end = i
        alter = df.iloc[:,start:end].fillna("").agg("".join,axis = 1)
        alter2 = df.drop(df.columns[start:end],axis = 1)
        pd.concat([alter , alter2] , axis = 1 )
        start = i

In [ ]:
get_valid_periods(df , header_map)

Passing periods: [(2, 29)]


[(2, 29)]

In [ ]:
rows

0     Name of Instrument  ISIN CodeIndustryYieldQuan...
1                       Equity & Equity related        
2      Listed/Awaiting listing on Stock Exchange       
3       Infosys Ltd.INE009A01021IT - Software 707726...
4       BHARTI AIRTEL LTD.INE397D01024Telecom - Serv...
5       TATA CONSULTANCY SERVICES LTD.INE467B01029IT...
6       Tech Mahindra Ltd.INE669C01036IT - Software ...
7       Zomato LtdINE758T01015Retailing 12450162743....
8       HCL Technologies Ltd.INE860A01027IT - Softwa...
9       Wipro Ltd.INE075A01022IT - Software 77788224...
10      Mphasis LtdINE356A01018IT - Software 6552218...
11      Persistent Systems LimitedINE262H01021IT - S...
12      SWIGGY LTDINE00H001014Retailing 3763781566.1...
13      Coforge LimitedINE591G01017IT - Software 171...
14      BHARTI HEXACOM LTD.INE343G01021Telecom - Ser...
15      Indus Towers Ltd.INE121J01017Telecom - Servi...
16      ZENSAR TECHNOLGIES LTD.INE520A01027IT - Soft...
17      Birlasoft LImitedINE836A01035IT - Softwa

In [140]:
import yaml
with open("config/amc_configs.yaml", "r") as f:
    data = yaml.safe_load(f)


In [141]:
for key,value in data.items():
    path = value["data_dir"]
    name = path.split("/")[-1]
    name = re.sub("_" , " " , name).title()
    value["data_dir"] = "/".join((path.split("/")[:-1] + [name]))
    path = value["output_file"]
    name = path.split("/")[-1]
    name = re.sub(".csv" , ".xlsx" , name)
    value["output_file"] = "/".join((path.split("/")[:-1] + [name]))

    

In [142]:
with open("config/amc_configs2.yaml","w") as f:
    yaml.dump(data,f ,default_flow_style=False)

In [126]:
"hello wolrd".title()

'Hello Wolrd'